# Vertex Covering Challenge

In [ ]:
from typing import Set, Tuple, Optional, List

from itertools import product

import numpy as np
from numpy import random as rand
from scipy.sparse import dok_matrix, csr_matrix

import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4

output_notebook()

In [ ]:
rand.seed(42)

In [ ]:
n_nodes = 5
n_edges = 5

n_edge_max = 3

In [ ]:
def generate_graph(n_nodes: int, n_edges: int, n_edge_max: int = 5) -> Set[Tuple[int]]:
    """Creates random edges for graph.
    
    Node labels are from 0 to n_nodes - 1 
    
    Arguments:
        n_nodes: Number of nodes
        n_edges: Number of edges
        n_edge_max: Maximal number of edges for graph
        
    Todo:
        What about connected / unconnected graphs?
        Am I creating the right graph? 
        Think about ill input conditions.
    """
    assert n_edge_max > 0
    assert n_nodes * n_edge_max // 2 > n_edges

    nodes = {n: 0 for n in range(n_nodes)}
    edges = set()

    while len(edges) < n_edges:
        choices = [n for n, count in nodes.items() if count <= n_edge_max]
        e1, e2 = (int(e) for e in rand.choice(choices, size=2, replace=False))
        edge = (e2, e1) if e2 < e1 else (e1, e2)
        if edge not in edges:
            edges.add(edge)
            nodes[e1] += 1
            nodes[e2] += 2

    return edges


graph = generate_graph(n_nodes, n_edges, n_edge_max=n_edge_max)

In [ ]:
def get_plot(graph: Set[Tuple[int]], color_nodes: Optional[List[int]] = None) -> Plot:
    """Creates bokeh graph plot from edge set.
    
    Arguments:
        graph: The graph to plot.
        color_nodes: Colors given nodes green, default is white.
    """
    color_nodes = color_nodes or []

    G = nx.Graph(list(graph))
    edge_attrs = {}
    node_attrs = {}

    for start_node, end_node, _ in G.edges(data=True):
        edge_attrs[(start_node, end_node)] = "black"

    for node in G.nodes:
        node_attrs[node] = "green" if node in color_nodes else "white"

    nx.set_edge_attributes(G, edge_attrs, "edge_color")
    nx.set_node_attributes(G, node_attrs, "node_color")

    # Show with Bokeh
    plot = Plot(
        plot_width=400,
        plot_height=400,
        x_range=Range1d(-1.1, 1.1),
        y_range=Range1d(-1.1, 1.1),
    )

    graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))
    graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="node_color")
    plot.renderers.append(graph_renderer)

    node_hover_tool = HoverTool(tooltips=[("index", "@index")])
    plot.add_tools(node_hover_tool)

    return plot


plot = get_plot(graph)
show(plot)

Form of the matrix which can be plugged in the annealer.

The condition to minimize is 

$$
    \sum_{i=0}^{N_n} x_i  \quad \Leftrightarrow \quad \vec x \cdot \mathbb{1} \cdot \vec x
$$

with $x_i = 1, 0$ specifying wethere a node is marked or not.

This condition needs to be minimized under the constraint

$$
    x_i + x_j \geq 1 \, \forall_{(i,j)} \in E
$$
where $E$ is the set of all edges.
This is equivalent ot saying that for all edges, at least one of their nodes is marked.

This constraint can be mapped to
$$
    x_i + x_j - s_{ij} - 1 = 0 \, , \quad s_{ij} = 0, 1 \, .
$$
The above equation can only be zero if at least on of $x_i, x_j = 1$.

Thus our D-Wave function is

$$
    \text{argmin}_{\vec x, \vec s}\left[
        \sum_{i=0}^{N_n-1} x_i
        + p
        \sum_{(i,j)=E} (x_i + x_j - s_{ij} - 1)^2 = 0
    \right]
    \, ,
$$
with $p > N_n$.

Since both $x$ and $s$ are 0,1 this is equal to

\begin{align}
    (x_i + x_j - s_{ij} - 1)^2
    &=
    x_i^2 + x_j^2 + s_{ij}^2 + 1^2
    + 2 x_i x_j - 2 x_i - 2 x_j
    - 2 s_{ij} x_i - 2 s_{ij} x_j + 2 s_{ij}
    \\&=
    - x_i^2 - x_j^2 + 2 x_i x_j + 3 s_{ij}^2 - 2 s_{ij} x_i - 2 s_{ij} x_j + 1
    \\&\to
    \vec \psi_j
    \left(
        - \delta_{i,i} - \delta_{j,j} + 2 \delta_{i,j}
        + 3 \delta_{ k_{ij}, k_{ij}}
        - 2 \delta_{ k_{ij}, i}
        - 2 \delta_{ k_{ij}, j}
    \right)
    \vec \psi_i
    + 1
\end{align}


This is implemented below for the vector $\vec \psi = (\vec x, \vec s)^T = (x_0, x_1, \cdots x_{N_n-1}, s_{12}, s_{13}, \cdots)$ this becomes



In [ ]:
def get_gc_matrix(n_nodes: int, graph: Set[Tuple[int]]) -> csr_matrix:
    """Implements the graph covering matrix.
    
    Arguments:
        n_nodes: Number of nodes
        graph: The graph.
    """
    n_edges = len(graph)
    mat = dok_matrix((n_nodes + n_edges, n_nodes + n_edges), dtype=int)

    for n in range(n_nodes):
        mat[n, n] = 1

    p = n_nodes + 1
    for s, (e1, e2) in enumerate(graph):
        mat[e1, e1] -= 1 * p
        mat[e2, e2] -= 1 * p
        mat[e1, e2] += 2 * p
        mat[n_nodes + s, e1] += -2 * p
        mat[n_nodes + s, e2] += -2 * p
        mat[n_nodes + s, n_nodes + s] += 3 * p

    return mat.tocsr()


mat = get_gc_matrix(n_nodes, graph)

In [ ]:
energies = []
vecs = []
for vv in product(*[(0, 1)] * mat.shape[1]):
    ee = vv @ mat @ vv
    energies.append(ee)
    vecs.append(vv)

e_min = np.min(energies)
solutions = [
    [nn for nn, bb in enumerate(vv[:n_nodes]) if bb == 1]
    for vv, ee in zip(vecs, energies)
    if ee == e_min
]
solutions

In [ ]:
for sol in solutions:
    plot = get_plot(graph, color_nodes=sol)
    show(plot)